In [160]:
import pandas as pd
import numpy as np
import sys
import os
import time
import pickle as pkl
import codecs
import re
import spacy

from amazon.api import AmazonAPI
from imdb import IMDb
from importlib import reload
from sklearn.preprocessing import MultiLabelBinarizer
import omdb

from IPython.display import clear_output

% matplotlib inline
pd.options.display.max_columns = 30

# 02B - Data Cleaning
In this notebook, we clean the data collected from Amazon, OMDB, and IMDB and, in some cases, prepare it such that it can easily be converted into a format a factorization machine can interpret.

In [147]:
data_path_1 = os.path.join('..','data')
data_path_2 = os.path.join('..','data-2')

df = pd.read_csv(os.path.join(data_path_1, 'reviews_sample_100.csv')).drop(['Unnamed: 0', 'reviewTime'], axis = 1)
df.columns = ['item', 'user', 'rating']
X_user_item = df.drop('rating', axis = 1)

# Amazon

Of the data collected from Amazon, we will keep only the following features we suspect may contribute predictive information on how a given user might rate an item:
- title
- asin
- genres
- actors
- directors
- studio
- sales rank
- running time

In [225]:
amazon_df_orig = pd.read_csv(os.path.join(data_path_2, 'amazon_data.csv'))
amazon_df_orig = amazon_df_orig[['title', 'asin', 'genre', 'actors','directors', 'studio', 'sales_rank', 'running_time']]
amazon_df = amazon_df_orig.copy()
amazon_df_orig.sample(4)

,title,asin,genre,actors,directors,studio,sales_rank,running_time
2628,Water (2 DVD Special Edition),B000E112BG,NaN,[Deepa Mehta],[],Mongrel,231964.0,115.0
1692,Son in Law [VHS],6302994071,NaN,"[Pauly Shore, Carla Gugino, Lane Smith, Cindy ...",[Steve Rash],Buena Vista,168473.0,96.0
3215,The Mark Of Zorro [VHS],6302700949,NaN,"[Tyrone Power, Linda Darnell, Basil Rathbone, ...",[Rouben Mamoulian],Fox Home Entertainme,210878.0,93.0
352,The Flintstones (Collector's Edition),0783231644,Music Videos & Concerts,"[John Goodman, Rick Moranis, Elizabeth Perkins...",[Brian Levant],Universal Studios Home Entertainment,6992.0,91.0


### Genre

For a factorization machine, a categorical feature like **genre** should be one-hot encoded. Unfortunately, as we see below, genre naming conventions are wildly inconsistent across Amazon's film and TV catalog. Furthermore, some items are classified as a specific combination of genres, which would be represented as an entire different genre under a strict one-hot encoding.

In [220]:
amazon_df_orig['genre'].unique()

array([nan, 'Drama', 'Westerns', 'Fantasy', 'Romance', 'Comedy',
       'Kids & Family', 'Anime & Manga', 'Musicals', 'Mystery & Thrillers',
       'Action & Adventure', 'Sports', 'Anime', 'COMEDY',
       'Science Fiction', 'Horror', 'Foreign', 'faith_&_spirituality',
       'Music Videos & Concerts', 'Military & War', 'Performing Arts',
       'Musical  Comedy', 'romantic-comedy', 'Documentary',
       'Folk Music - General', 'Gay & Lesbian Video',
       'PB ACTION/ADVENTURE', 'Children', 'ACTION/ADVENTURE', 'comedy',
       'Exercise & Fitness', 'CHILDREN/FAMILY', 'Aliens', 'PB WESTERNS',
       'PB COMEDY', 'PB DRAMA', 'Talk Shows', 'WESTERNS', 'HORROR',
       'DRAMA', 'Dramas', 'foreign', 'DVD', 'Drama, Horror',
       'Drama, Sports,', 'drama', 'SCIENCE FICTION', 'Biographical',
       'Medical Drama', 'Biography', 'Religion', 'TV',
       'Faith & Spirituality', 'action-game-genre', 'TV Shows',
       'mystery-and-suspense', 'MYSTERY/SUSPENSE',
       'kids_&_family comedy act

To account for the problems listed above, we can think of **genre** less as a categorical feature and more as a short string to be tokenized and later converted to a bag-of-words vector. This allows films with multiple genres to be counted as such and mitigates the effects of artificial distinctions like "fantasy action" vs "action fantasy." Furthermore, we apply lemmatization to all tokens so that distinctions like 'romance' vs 'romantic' are also eliminated.

In [230]:
# load english language model for lemmatization
nlp = spacy.load('EN')

def tokenize_genre(genre_string):
    if isinstance(genre_string, float):
        return []
    else:
        # find all contiguous sequences of alphanumeric characters
        regex = re.compile(r'(\w+)')
        result = re.findall(regex, genre_string)
        
        # merge tokens into single line       
        genre_string = ''
        for token in result:
            # convert token to lower case, remove '_', and add trailing whitespace
            token = token.lower().replace('_', '') + ' '
            genre_string = genre_string + token
        
        # lemmatize all tokens
        tokens = nlp(genre_string.strip())
        tokens = [x.lemma_ for x in tokens]

        return tokens

Below we confirm that the tokenization procedure is successful. The actual conversion of this feature to one-hot encoding will be performed in a later notebook.

In [239]:
amazon_df['genre'] = amazon_df_orig['genre'].apply(tokenize_genre)
amazon_df['genre'].sample(5)

6041                        [military, war]
1704                    [action, adventure]
3640                                     []
5773    [drama, fantasy, action, adventure]
678                     [mystery, thriller]
Name: genre, dtype: object

### Actors

All data on actors and directors are stored in strings that resemble lists.

In [250]:
amazon_df_orig.loc[:,'actors'][0]

'[Kristen Wiig, Matt Dillon, Darren Criss]'

Below, we convert the all actor strings to lists. In the process, we also reduce all text to lower case.

In [253]:
def tokenize_person(actor_string):
    string = actor_string[1:-1]
    tokens = string.split(',')
    tokens = [x.lower().strip() for x in tokens]
    return tokens

amazon_df['actors'] = amazon_df_orig['actors'].apply(tokenize_person)
print(amazon_df['actors'].sample(5))
print()
amazon_df['directors'] = amazon_df_orig['directors'].apply(tokenize_person)
print(amazon_df['directors'].sample(5))

1250    [jobeth williams, craig t. nelson, heather o'r...
5774    [sarah gellar, kadee strickland, bill pullman,...
5300    [cate blanchett, giovanni ribisi, remo girone,...
4317                                        [ilona elkin]
6260    [nigel hawthorne, helen mirren, rupert graves,...
Name: actors, dtype: object

3448                 []
5180    [fred schepisi]
3840     [peter hewitt]
1369    [michael apted]
4167       [peter berg]
Name: directors, dtype: object


### Studio
Similarly to **genres**, production studio names are also assigned inconsistently (i.e. Warner Bros. vs Warner Brothers vs Warner Home Entertainment). The solution is also similar to **genres**, so we apply the same function for processing.

In [256]:
amazon_df['studio'] = amazon_df_orig['studio'].apply(tokenize_genre)
amazon_df['studio'].sample(5)

2911                        [usa]
7037       [mgm, ua, home, video]
3314        [warner, home, video]
1318     [showbox, entertainment]
1173    [buena, vista, home, ent]
Name: studio, dtype: object

In [266]:
amazon_df.sample(5)

,title,asin,genre,actors,directors,studio,sales_rank,running_time
5721,Scream 3,6305892946,[],"[david arquette, neve campbell, courteney cox,...",[],[],537591.0,116.0
5715,The Shining [VHS],0790705141,[documentary],"[jack nicholson, shelley duvall, danny lloyd, ...",[stanley kubrick],"[warner, home, video]",48052.0,144.0
5488,Flightplan (Full Screen Edition),B000BYY11O,"[mystery, thriller]","[jodie foster, peter sarsgaard, erika christen...",[robert schwentke],"[buena, vista, home, entertainment, touchstone]",66750.0,98.0
280,Jesse Stone: No Remorse,B003ES5JJ2,[],"[tom selleck, kathy baker, william devane]",[],[sony],20230.0,87.0
4784,Cars (Single-Disc Widescreen Edition),B00005JNS0,"[kid, family]","[owen wilson, paul newman, bonnie hunt, rodger...",[john lasseter],"[disney, pixar]",98.0,117.0


# OMDB

In [351]:
omdb_df_orig = pd.read_csv(os.path.join(data_path_2, 'omdb_data.csv'))
omdb_df = omdb_df_orig.drop('writer', axis = 1)
omdb_df.sample(3)

,asin,title,actors,box_office,country,director,genre,language,metascore,production,rated,released,runtime,type,year,imdb_id,imdb_rating,imdb_votes
3129,B001URA5Y8,Powder Blue,"Jessica Biel, Eddie Redmayne, Forest Whitaker,...",NaN,USA,Timothy Linh Bui,Drama,English,NaN,Speakeasy Releasing,R,15 Oct 2009,106 min,movie,2009,tt1032819,6.3,"12,883"
4568,0783219660,Sudden Death,"Jean-Claude Van Damme, Powers Boothe, Raymond ...",NaN,USA,Peter Hyams,"Action, Crime, Thriller",English,NaN,MCA Universal Home Video,R,22 Dec 1995,111 min,movie,1995,tt0114576,5.7,"27,627"
2425,B001CIOCJY,Sukiyaki Western Django,"Hideaki Itô, Masanobu Andô, Kôichi Satô, Kaori...",NaN,Japan,Takashi Miike,"Action, Western",English,55.0,First Look Studios,R,15 Sep 2007,121 min,movie,2007,tt0906665,6.3,"13,416"


### Actors, Directors, Language, Country

In [399]:
def tokenize_basic_omdb(string):
    if isinstance(string, str):
        tokens = string.split(',')
        tokens = [x.lower().strip() for x in tokens]
    else:
        tokens = list()
    return tokens

omdb_df['actors'] = omdb_df_orig['actors'].apply(tokenize_basic_omdb)
omdb_df['director'] = omdb_df_orig['director'].apply(tokenize_basic_omdb)
omdb_df['language'] = omdb_df_orig['language'].apply(tokenize_basic_omdb)
omdb_df['country'] = omdb_df_orig['country'].apply(tokenize_basic_omdb)

omdb_df[['actors', 'director', 'language', 'country']].sample(5)

,actors,director,language,country
6400,"[tom hanks, sarah mahoney, roxana ortega, rand...",[tom hanks],[english],[usa]
1057,[],[nick park],[english],[uk]
609,"[val kilmer, alexandra staseson, brad dryborou...",[mark a. lewis],[english],[canada]
19,"[ewan mcgregor, christopher plummer, mélanie l...",[mike mills],"[english, french]",[usa]
5401,"[ivana baquero, sergi lópez, maribel verdú, do...",[guillermo del toro],[spanish],"[spain, mexico, usa]"


### Genre, Studio

In [325]:
def tokenize_lemma_omdb(string):
    if isinstance(string, str):
        string = re.sub(pattern=r',|/|-|/.', repl='', string=string)
        string = string.replace('.', '')
        string = string.lower()
        tokens = nlp(string)
        tokens = [x.lemma_ for x in tokens]
    else:
        tokens = list()
    return tokens

omdb_df['genre'] = omdb_df_orig['genre'].apply(tokenize_lemma_omdb)
omdb_df['production'] = omdb_df_orig['production'].apply(tokenize_lemma_omdb)

omdb_df[['genre','production']].sample(5)

,genre,production
339,"[action, adventure, family]","[buena, vista]"
1422,"[horror, thriller]","[lionsgate, entertainment]"
3506,[documentary],[]
6049,"[biography, drama, sport]","[walt, disney, picture]"
4339,"[family, fantasy]","[twentieth, century, fox]"


### MPAA Rating

The MPAA and TV ratings are consistently applied, with the exception of 'NOT RATED', 'UNRATED', 'Unrated', and 'nan', which are better collapsed into one rating. We replace all such ratings with missing values.

In [491]:
def convert_mpaa_rating(string):
    no_rating = ['NOT RATED', 'UNRATED', 'Unrated']
    if isinstance(string, float):
        return np.nan
    else:
        if string in no_rating:
            return np.nan
        else:
            return string

omdb_df['rated'] = omdb_df_orig['rated'].apply(convert_mpaa_rating)
omdb_df['rated'].value_counts()

R           2488
PG-13       1454
PG           929
APPROVED     369
G            195
PASSED        62
TV-14         51
TV-PG         39
TV-MA         37
GP            23
NC-17         20
TV-G          12
M              7
X              5
TV-Y7          2
TV-Y           2
M/PG           2
E              1
Name: rated, dtype: int64

### Box office

Box office data is stored as strings, which we convert to floats.

In [481]:
def convert_box_office(string):
    if not isinstance(string, float):
        string = string.replace('$', '')
        string = string.replace(',', '')
        return float(string)

omdb_df['box_office'] = omdb_df_orig['box_office'].apply(convert_box_office)
omdb_df['box_office'].sample(10)

2335     19976073.0
2366     28687835.0
6657            NaN
2207            NaN
3521            NaN
5167    134294280.0
1412            NaN
1763            NaN
4134            NaN
6507            NaN
Name: box_office, dtype: float64

### Runtime

In [361]:
def convert_runtime(string):
    if not isinstance(string, float):
        string = string.replace('min', '')
        string = string.strip()
        return float(string)

omdb_df['runtime'] = omdb_df_orig['runtime'].apply(convert_runtime)
omdb_df['runtime'].sample(5)

954     100.0
2726     84.0
4879    130.0
5170     91.0
246      66.0
Name: runtime, dtype: float64

In [388]:
def convert_votes(string):
    if not isinstance(string, float):
        string = string.replace(',', '')
        return float(string)

omdb_df['imdb_votes'] = omdb_df_orig['imdb_votes'].apply(convert_votes)
omdb_df['imdb_votes'].sample(5)

5695    11168.0
3177    21198.0
1114    44622.0
263     17157.0
4122    86914.0
Name: imdb_votes, dtype: float64

# IMDB

In [363]:
imdb_df_orig = pd.read_csv(os.path.join(data_path_2, 'imdb_data.csv'))
imdb_df = imdb_df_orig.copy()
imdb_df.sample(3)

,title,asin,year,vfx,genres,runtimes,production_companies,rating,votes,directors
4376,Fur: An Imaginary Portrait of Diane Arbus,B000NO1XG8,2006,True,"['Biography', 'Drama', 'Romance']",122,"['Edward R. Pressman Film', 'River Road Entert...",6.5,13916.0,['Steven Shainberg']
2049,The Mummy,6302814707,1999,True,"['Action', 'Adventure', 'Fantasy']",124,"['Universal Pictures', 'Alphaville Films']",7.0,343144.0,['Stephen Sommers']
6720,Prophecy,6304707045,1979,True,"['Sci-Fi', 'Horror']",102,['Paramount Pictures'],5.4,3312.0,['John Frankenheimer']


### Genres, Directors

In [377]:
def convert_imdb_basic(string):
    if not isinstance(string, float):
        in_list = eval(string)
        out_list = [x.lower() for x in in_list]
        return out_list

imdb_df['genres'] = imdb_df_orig['genres'].apply(convert_imdb_basic)
imdb_df['directors'] = imdb_df_orig['directors'].apply(convert_imdb_basic)
imdb_df[['genres', 'directors']].sample(10)

,genres,directors
1785,"[comedy, drama, romance]",[garry marshall]
1293,"[comedy, horror]",[karyn kusama]
3510,"[action, crime, horror, mystery]",[william lustig]
1855,"[horror, thriller]",[roger spottiswoode]
163,"[comedy, drama, family, music, musical, romance]",[adam shankman]
960,"[adventure, drama, western]",[john ford]
4835,"[horror, thriller]",[nick murphy]
4605,"[drama, family, fantasy, sport]",[phil alden robinson]
1246,"[crime, drama, thriller]",[paul haggis]
6405,"[drama, mystery, romance, thriller]",[neil burger]


### Production companies

In [381]:
def convert_imdb_lemma(string):
    if not isinstance(string, float):
        in_list = eval(string)
        string = ''
        for token in in_list:
            token = token.lower()
            token = re.sub(pattern=r',|/|-|/.', repl='', string=token)
            token = token.replace('.', '')
            string += (token + ' ')
        tokens = nlp(string)
        out_list = [x.lemma_ for x in tokens]
        return out_list
    
imdb_df['production_companies'] = imdb_df_orig['production_companies'].apply(convert_imdb_lemma)
imdb_df['production_companies'].sample(5)

1955    [evolution, entertainment, saw, production, in...
6316                                [touchstone, picture]
583     [twentieth, century, fox, mediastream, vierte,...
3021                                 [hal, roach, studio]
6062    [polygram, film, entertainment, egg, picture, ...
Name: production_companies, dtype: object

# Comparing similar features from different sources

The data from Amazon, OMDB, and IMDB contain overlapping features. In many cases, we need to choose just one feature among duplicates.

### IMDB votes and ratings
Unsurprisingly, votes and ratings from IMDB more up to date than those from OMDB. Furthermore, IMDB vote and rating data has fewer missing values than OMDB, so we will drop OMDB vote/rating data in favor of those from IMDB.

In [396]:
print('imdb vote count nulls:', imdb_df['votes'].isnull().sum())
print('omdb vote count nulls:', omdb_df['imdb_votes'].isnull().sum())
print('imdb rating nulls:', imdb_df['rating'].isnull().sum())
print('omdb rating nulls:', omdb_df['imdb_rating'].isnull().sum())
pd.merge(imdb_df[['asin', 'votes']], omdb_df[['asin', 'imdb_votes']], how='inner', on = 'asin').sample(10)

imdb vote count nulls: 159
omdb vote count nulls: 167
imdb rating nulls: 159
omdb rating nulls: 166


,asin,votes,imdb_votes
1237,630340720X,7336.0,7305.0
1776,B008220C92,179975.0,179661.0
4323,B005F96UNM,17548.0,17541.0
6713,B000078UJW,NaN,NaN
563,B00003CXKM,84805.0,84542.0
3448,6302256682,1237.0,1234.0
4027,B002VECLVO,93440.0,93160.0
3004,B0059XTV4Y,10586.0,10579.0
4605,0783225881,92685.0,92317.0
3889,B00003CXWL,46501.0,46461.0


### Directors
There is only one instance where data on directors from IMDB and OMDB do not match. 

In [402]:
merged = pd.merge(imdb_df[['asin', 'directors']], omdb_df[['asin', 'director']], how='inner', on = 'asin').sample(10)
merged[merged['directors'] != merged['director']]

,asin,directors,director
1066,6302643627,[franco zeffirelli],[]


There are almost 2000 instances where director data from Amazon and IMDB do not match, oftentimes because Amazon has no data. Because IMDB data appears more comprehensive and more consistently labeled, we choose to keep director data only from IMDB.

In [412]:
merged = pd.merge(amazon_df[['asin', 'directors']], imdb_df[['asin', 'directors']], how='inner', on = 'asin')
merged[merged['directors_x'] != merged['directors_y']].sample(10)

,asin,directors_x,directors_y
3457,1415719284,[],[cameron crowe]
6204,B00005JL98,[],[nicholas meyer]
6072,B004EPYZP8,[],[brad furman]
2011,B005F3XV62,[],[frank coraci]
6717,B00005JPAM,[],[stephen frears]
5883,630569270X,[],[p.j. pesce]
6528,B00DCLT8RA,[],[anthony c. ferrante]
5491,6305229449,"[barry cook, tony bancroft]","[tony bancroft, barry cook]"
1814,B00003CXOF,[],[janusz kaminski]
2787,B000FA57WG,[joe dante],[morgan j. freeman]


### Actors

The quality of data on actors appears roughly equivalent between Amazon and OMDB. Because Amazon's data appears more comprehensive here, we choose Amazon's data.

In [435]:
print('# actors listed per movie, OMDB\n\n', omdb_df['actors'].apply(lambda x: len(x)).value_counts())
print()
print('# actors listed per movie, Amazon\n\n', amazon_df['actors'].apply(lambda x: len(x)).value_counts())
merged = pd.merge(amazon_df[['asin', 'actors']], omdb_df[['asin', 'actors']], how='inner', on = 'asin')
merged[merged['actors_x'] != merged['actors_y']].sample(10)

# actors listed per movie, OMDB

 4    6505
0      90
1      69
2      54
3      51
Name: actors, dtype: int64

# actors listed per movie, Amazon

 5    5721
2     536
1     518
4     284
3     274
Name: actors, dtype: int64


,asin,actors_x,actors_y
5130,0790729601,"[arnold schwarzenegger, vanessa williams, jame...","[arnold schwarzenegger, james caan, vanessa wi..."
6126,B00BEIYLO8,"[mark wahlberg, dwayne johnson]","[mark wahlberg, dwayne johnson, anthony mackie..."
4861,B009AMALGM,"[katie featherston, kathryn newton]","[katie featherston, kathryn newton, matt shive..."
2736,B00004RERO,"[rosalind ayres, roy barraclough, tim barrett,...","[richard chamberlain, gemma craven, annette cr..."
988,B000A3DGEY,"[jeremy sisto, erik palladino, bianca lawson, ...","[andrea daveline, michael ray davis, adam fort..."
1374,B00C7BZY4C,"[michael shannon, chris evans]","[michael shannon, winona ryder, chris evans, r..."
4788,B002ZG97PG,"[russell brand, jonah hill, rose byrne]","[russell brand, rose byrne, tyler mckinney, zo..."
6345,B001E95ZNS,[anne hathaway],"[sebastian stan, roslyn ruff, anne hathaway, b..."
6078,B001LLH8S4,"[derek luke, michael ealy, laz alonso, omar be...","[derek luke, michael ealy, laz alonso, omar be..."
2121,B000A1IOGG,"[jennifer jason leigh, ellen barkin, stephen a...","[matthew faber, angela pietropinto, bill buell..."


### Runtimes
Between OMDB and IMDB data, runtimes from OMDB are more consistently formatted.

In [443]:
pd.merge(omdb_df[['asin', 'runtime']], imdb_df[['asin','runtimes']]).sample(10)

,asin,runtime,runtimes
6524,0792838416,115.0,115
5062,079284596X,106.0,106
2468,B00275EHFU,90.0,90
4801,6301606892,113.0,113
1848,6301802330,110.0,110
2163,6301978013,128.0,128
3268,0790740060,104.0,104
5763,B0010DR4BO,160.0,160
4054,B0034GK78C,88.0,USA:88
3641,B000YDBP3O,95.0,95


Runtimes between Amazon and OMDB largely match, but OMDB runtimes have fewer missing values.

In [457]:
print('omdb runtime nulls:', omdb_df['runtime'].isnull().sum())
print('amazon runtime nulls:', amazon_df['running_time'].isnull().sum())
pd.merge(omdb_df[['asin', 'runtime']], amazon_df[['asin','running_time']]).sample(10)

omdb runtime nulls: 166
amazon runtime nulls: 344


,asin,runtime,running_time
4556,B00000I1NO,94.0,94.0
343,B000078UJV,104.0,104.0
125,B000EXDS7K,NaN,286.0
104,B003UESJLU,97.0,98.0
685,B002BVYBJW,50.0,300.0
3871,779913027X,94.0,94.0
4265,630321245X,134.0,134.0
3549,B0016MJ6JM,101.0,101.0
1645,0783240031,95.0,NaN
2889,B004EPZ0AC,100.0,100.0


### Genre and Production Company
Regarding both features, data from IMDB is much more comprehensive than that from Amazon. However, because the type of data Amazon provides for both features is significantly different (and much leaner), there may be value in keeping both sets of data for these two features.

In [467]:
pd.merge(amazon_df[['asin', 'genre']], imdb_df[['asin', 'genres']], how='inner', on='asin').sample(20)

,asin,genre,genres
1758,6303662544,[],"[drama, romance]"
2990,0790732254,[],[drama]
4012,6304435096,[],"[horror, thriller]"
3255,B0000VV4OK,[comedy],"[comedy, drama]"
2956,B000NIVJFO,[],"[comedy, drama, romance]"
3107,6300271439,[],"[comedy, music, romance]"
4942,B00005JM20,[],"[action, adventure, comedy]"
2752,B00005J760,[],"[drama, history, thriller]"
3603,6303935257,[romance],"[comedy, drama, romance]"
3575,B000E5KQOQ,[],"[adventure, drama]"


In [472]:
pd.merge(amazon_df[['asin', 'studio']], imdb_df[['asin', 'production_companies']], how='inner', on='asin').sample(20)

,asin,studio,production_companies
4789,079072961X,"[warner, home, video]","[warner, bro, kopelson, entertainment]"
5839,B000CCW2OS,"[20th, century, fox]","[fox, 2000, picture, scott, free, production, ..."
5214,B00030590I,[miramax],"[beij, new, picture, film, co, china, film, co..."
4202,0780656946,[],"[new, line, cinema, platinum, dune, next, ente..."
1863,6303393985,"[20th, century, fox]","[twentieth, century, fox]"
5646,B000MCH5P4,[paramount],"[paramount, picture, paramount, vantage, anony..."
1774,B0015QWZ8S,"[twentieth, century, fox]",None
5433,B0000YTOKI,"[miramax, home, entertainment]","[miramax, red, hour, film, flower, film, kalis..."
3005,B00005JNQA,"[universal, studio, home, entertainment]","[universal, picture, work, title, film, misher..."
3429,6301763041,"[universal, studio]","[universal, picture, imagine, entertainment]"


# Building a common dataset
Finally, we build a dataset combining relevant data from all three sources. We perform an inner merge on the data using each item's ASIN, thus removing all items from the dataset that were not successfully queried from OMDB or IMDB. Such an operation will reduce the number of items from **7,333** to **6765**.

In [493]:
# subset features from Amazon
amazon_subset_df = amazon_df[['asin', 'genre', 'actors', 'studio', 'sales_rank']]
amazon_subset_df.columns = ['asin', 'genres_amazon', 'actors', 'studios_amazon', 'sales_rank']

# subset features from OMDB
omdb_subset_df = omdb_df[['asin', 'title', 'box_office', 'country', 'language', 'metascore', 'rated', 'runtime',
                         'type', 'year']]
omdb_subset_df.columns = ['asin', 'title', 'box_office', 'country', 'language', 'metascore', 'mpaa_rating', 'runtime',
                         'type', 'year']

# subset features from IMDB
imdb_subset_df = imdb_df[['asin', 'vfx', 'genres', 'production_companies', 'rating', 'votes', 'directors']]
imdb_subset_df.columns = ['asin', 'vfx', 'genres_imdb', 'studios_imdb', 'imdb_rating', 'imdb_votes', 'directors']

merge1_df = pd.merge(omdb_subset_df, imdb_subset_df, on='asin', how='inner')
merged_df = pd.merge(merge1_df, amazon_subset_df, on='asin', how='inner')

In [494]:
print(merged_df.shape)
merged_df.sample(5)

(6765, 20)


,asin,title,box_office,country,language,metascore,mpaa_rating,runtime,type,year,vfx,genres_imdb,studios_imdb,imdb_rating,imdb_votes,directors,genres_amazon,actors,studios_amazon,sales_rank
3202,6303111874,Getaway,10494494.0,"[usa, bulgaria]",[english],22.0,PG-13,90.0,movie,2013,True,"[action, crime, thriller]","[after, dark, film, dark, castle, entertainmen...",4.4,20915.0,[courtney solomon],[],"[steve mcqueen, ali macgraw, ben johnson, sall...","[warner, home, video]",268312.0
5195,6300251330,Saturn 3,NaN,[uk],[english],NaN,R,96.0,movie,1980,True,"[adventure, horror, sci-fi, thriller]","[itc, film, elliott, kastner, production, tran...",5.1,6735.0,"[stanley donen, john barry]","[action, adventure]","[farrah fawcett, kirk douglas, harvey keitel, ...","[20th, century, fox]",325221.0
6019,6300151379,Labyrinth,NaN,"[uk, usa]",[english],50.0,PG,101.0,movie,1986,True,"[adventure, family, fantasy, musical]","[henson, associate, (, ha, ), lucasfilm, jim, ...",7.4,103800.0,[jim henson],[fantasy],"[david bowie, jennifer connelly, toby froud, s...","[nelson, entertainment]",173463.0
4847,6302909651,Sliver,NaN,[usa],[english],NaN,R,107.0,movie,1993,True,"[drama, thriller]","[paramount, picture, robert, evan, company]",4.9,24620.0,[phillip noyce],[],"[sharon stone, william baldwin, tom berenger, ...",[paramount],108935.0
121,6303102492,Pinky,NaN,[usa],[english],NaN,APPROVED,102.0,movie,1949,False,[drama],"[twentieth, century, fox]",7.2,2134.0,"[elia kazan, john ford]",[],"[jeanne crain, ethel barrymore, ethel waters, ...","[20th, century, fox]",268636.0


In [495]:
with open(os.path.join(data_path_2, 'item_data.df'), 'wb') as file_out:
    pkl.dump(merged_df, file_out)